## Del 1: Maskinoversettelse 
#### Laget av Emma, Morten og Mark

Vi skal bruke en (nevral) maskinoversettelsemodell til å oversette filmtekstinger fra Ringenes Herre (og Hobbiten) fra tysk til engelsk.

<b>NB:</b> Vi har skrevet ferdig kode på python filen, det vil si at vi importerer den når vi skal ta i bruk alle metoder, osv...

#### Komme i gang

<b>Viktig:</b> oversettelsmodeller krever at alle ordene med små bokstaver. Derfor må vi forvandle alle linjene til lowercase. Vi har brukt følgende python script for å få til alle filer vi trenger for del 1:

In [1]:
with open("lotr.de") as fd_in, open("lotr.lc.de", "w") as fd_out:
    for line in fd_in:
        fd_out.write(line.lower())

with open("lotr.lc.de") as fd_in_en, open("lotr.small.lc.de", "w") as fd_out_to:
    for line in fd_in_en:
        fd_out_to.write(line.lower())


Etter at vi har forvandlet alle linjene til lowercase, lager vi en liten testfil med de første 100 linjene av filmtekstingene. Vi bruker kommandoen som er gitt i obligen, det vil si: <br> <br>
<b> $ head -n 100 lotr.lc.de > lotr.small.lc.de. </b> <br><br>
Som en konsekvens, kjører vi OpenNMT på filen. 

<b> $ onmt_translate --batch_size 5 --beam_size 3 --model de2en.pt --src lotr.small.lc.de --output lotr.small.out.en </b> <br> <br> Som en liten test, kjører vi følgende kode for å se om resultatet stemmer over det som er blitt gitt på oblig-pdfen. Det er viktig å merke seg at vi gjør samme prosess for del 1, det vil si at i stedenfor 100 linjer, så tar vi med alle setninger som er i filen.  

In [2]:
sentences = open("lotr.small.out.en", "r")
sentence = sentences.readlines()
for x in range(7):
    print(sentence[x])

the world is changing .

i feel it in the water .

i feel it in the world .

i smell it in the air .

a lot of what once was , is lost because nobody lives more , who remembers .

it started with the back of the big rings .

three were given the elben , the immortal , weisesten and reinsten of all creatures .



Et ting som vi har legget merket til, og som også nevnes i obligen, er at det finnes noen <b>"spesielle"</b>ord, nemlig "unk"-ordene, som viser at modellen ikke har klart å oversette noen tyske ord. <br>

Setningen på engelsk ser ut sånn: <br><br>
<b>three were given the "unk" , the immortal , "unk" and "unk" of all creatures .</b> <br><br>
Mens setningen på tysk er nemlig: <br><br>
<b>3 wurden den Elben gegeben , den unsterblichen , weisesten und reinsten aller Wesen . </b> <br><br>

OpenNMT tilbyr da en måte å erstatte ukjente ord med ordene i kildespråket (altså tysk) som har høyest "attention weight" ved generering av hvert "unk"-ord. Mekanisme aktiveres ved å legge til <b>-replace_unk:</b><br><br>
<b>$ onmt_translate --batch_size 5 --beam_size 3  --model de2en.pt --src lotr.small.lc.de --output lotr.small.out.en --replace_unk</b>

In [3]:
# Sjekker hva vi får på linje 7 en gang til.
sentences = open("lotr.small.out.en", "r")
sentence = sentences.readlines()
print(sentence[6])

three were given the elben , the immortal , weisesten and reinsten of all creatures .



#### a) Utvikling av en frasetabell (2 poeng)

Vi skal nå sørge for at systemet blir bedre på å oversette ukjente ord som "Elben". OpenNMT tilbyr da en postprosessringsmekanisme for å håndtere ukjente ord med en såkalt frasetabell, altså en slags ordbok. Før vi tok i bruk kommandoen som tar ordboken og bytter ordene mellom tysk og engelsk, ville vi se hvordan setningene så ut før vi konverterte dem. Vi tok i bruk følgende kode-snitt og fikk følgende setninger: 

In [4]:
# Det er verdt å nevne at vi markerte alle ord som kunne ikke oversettes med svart. 
# På denne måten kunne vi kjøre kommandoen og se forskjellen. 
sentences = open("lotr.small.out.en", "r")
sentence = sentences.readlines()
for x in range(9, 15):
    print(str(x + 1) + " " + sentence[x])

10 because these rings bargen the force and the will to lead every people .

11 but they all cheated .

12 because a ring was made .

13 in mordor , in the feuer of the schicksalsberges schmiedete of the dark herrscher sauron , the dark herrscher sauron heimlich a meisterring to beherrschen everyone else .

14 in this ring , his cruelty , his bosheit and his will , went to unterdrücken all life .

15 a ring to knechten them .



10 because these rings are <b>bargen</b> the force and the will to lead every people . <br>

11 but they all cheated . <br>

12 because a ring was made . <br>

13 in mordor , in the  <b>feuer</b> of the  <b>schicksalsberges schmiedete</b>  of the dark  <b>herrscher</b> sauron , the dark  <b>herrscher</b> sauron  <b>heimlich</b> a  <b>meisterring</b> to  <b>beherrschen</b> everyone else . <br>

14 in this ring , his cruelty , his  <b>bosheit</b> and his will , would  <b>unterdrücken</b> all life . <br>

15 a ring to  <b>knechten</b> them . <br>

#### Kommandoen før vi begynner å løse oppgaven: 

Nå som vi vet hva vi skal gjøre videre, kan vi da kjøre følgende kommando og svare på spørsmålene: <br>
$ onmt_translate --batch_size 5 --beam_size 3 --model de2en.pt --src lotr.small.lc.de --output lotr.small.out_with_dic.en --replace_unk --phrase_table de-en.txt

### Oppgaver: 

I dette tilfelle bruker vi samme kode som ovenfor, med det eneste forskjellen er at vi bruker den filen som ble generert denne gangen, altså filen som inneholder setningen oversatt med ordboken. Her er setningene: <br>

#### 1. Hva er modellens oversettelse for linjene 10-15?

In [10]:
sentences = open("lotr.small.out_with_dic.en", "r")
sentence = sentences.readlines()
for x in range(9, 15):
   print(str(x + 1) + " " + sentence[x])

10 because these rings bargen the force and the will to lead every people .

11 but they all cheated .

12 because a ring was made .

13 in mordor , in the fieriness of the bmount doom schmiedete of the dark lord sauron , the dark lord sauron backdoor a meisterring to mastery everyone else .

14 in this ring , his cruelty , his cattiness and his will , went to bottle up all life .

15 a ring to enslave them .



PRED AVG SCORE: -0.5285, PRED PPL: 1.6964

før oversettelse med ordbok: <br>
10 because these rings are bargen the force and the will to lead every people . <br>
11 but they all cheated .<br>
12 because a ring was made .<br>
13 in mordor , in the feuer of the schicksalsberges schmiedete of the dark herrscher sauron , the dark herrscher sauron heimlich a meisterring to beherrschen everyone else .<br>
14 in this ring , his cruelty , his bosheit and his will , would unterdrücken all life .<br>
15 a ring to knechten them .<br>

etter oversettelse:<br>
10 because these rings are bargen the force and the will to lead every people .<br>
11 but they all cheated .<br>
12 because a ring was made .<br>
13 in mordor , in the fieriness of the schicksalsberges schmiedete of the dark lord sauron , the dark lord sauron backdoor a meisterring to mastery everyone else . <br>
14 in this ring , his cruelty , his cattiness and his will , would bottle up all life .<br>
15 a ring to enslave them .<br>

#### 2. Finn et eksempel hvor bruk av ordboken førte til en dårlig oversettelse.

I linje 74 står det i den engelske oversettelsen "Oh, the long-expected party ." 
Dette har med orboken blitt oversatt til " the langerwartete is solid ." 
ordet "langewartete" på tysk kan bli oversatt til engelsk til "long awaited" eller "long-expected".
Setningen i seg selv, dersom vi også oversetter ordet, gir ikke mening. Vil derfor konkludere med at dette 
er et eksempel der oversettelsen er realtivt dårlig ved bruk av ordboken.

Et eksempel som vi kunne finne hvor bruken av ordboken førte til en dårlig oversettelse kan være i linje 13, hvor den klarte ikke helt å oversette det som tidligere var tyske ord, nemlig, "schicksalsberges schmiedete". 

#### 3. Lage en liste med minst 10 ord som systematisk er feil oversatt (som f.eks. “Beutlin” som bør oversettes til “Baggins”) og legge disse i frasetabellen de-en.txt.

Det vi gjorde er å printe alle setninger som ligger i filene <b>"lotr.small.out.en"</b> og <b> "lotr.small.out_with_dic.en" </b>. Med dette, var vi i stand om setningen forandret seg, altså om den tyske ordet ble forandre til et engelsk ord. Hvis dette ikke er tilfelle, fant vi en oversettelse av ordet og satt det i filen som inneholder tyske-engelsk ord. Her er en liste over ordene: <br>
beutlin - Baggins <br>
elben - elves <br>
handverkern - craftsmen <br>
auenland - shire <br>
schicksalsberges - mount doom <br>
herrscher - lord <br>
ringes - ring  <br>
!langewartete - long awaited <br> 
giltst - labeled <br>
weisesten - wisest <br>
reinsten - purest <br>

### b) Evaluering (4 poeng)

I denne delen er nå vi klare til å evaluere kvaliteten på oversettelsene vi har generert. Her bruker vi en evalueringsmetode som er veldig populær i maskinoversettelse, nemlig BLEU. 

#### Oppgaver

In [11]:
import oblig2b as maskin

#### 1. Fyll ut resten av funksjonen compute_precision(ref_file, output_file, ngram_order) som beregner precision-verdien (som definert over) for en gitt N-gram ordre. 

(Sjekk .py filen for compute_precision funksjonen

#### 2. Kjør OpenNMT på hele filen lotr.de (husk å konvertere den til lowercase først) uten frasetabell og beregne BLEU-scoren ved å kalle funksjonen compute_bleu(ref_file, output_file), som allerede er implementert.

In [12]:
print(maskin.compute_bleu("lotr.en", "lotr.small.out.en"))

0.22995760630328946


Det er verdt å nevne at det tok oss veldig langt å skjønne oppgaven. Etter lange netter med ulike type tester og lesing, klarte vi å komme med en metode som ga oss akkurat det tallet dere letter nemlig, 0.229.. Dette har vi klart ved å bruke filene vi genererte med "kommandoene" fra obligen. 

#### 3. Kjør OpenNMT på hele filen lotr.de, denne gang med frasetabellen som du har redigert, og beregne BLEU-scoren.

In [13]:
#print(maskin.compute_bleu("lotr.en", "lotr.small.out_with_dic.en"))

Følgende funksjon gir oss et resultat på 0.23367... I obligen står det at det skal ligge mellom 0.235 og 0.24, så her her vi bommet med 0.002 %, men vi regner med at det ikke gjør en så stor forksjell, men ellers så har vi regnet det dere lettet etter. 

## Del 2: Interaktive systemer (4 poeng)

I denne delen av oppgaven skal vi utvikle en liten "retrieval-based chatbot" basert på korpuset i "lotr.en", altså den vi brukte i forrige del-oppgave.

Før vi satt oss med denne oppgaven, lagde vi en liste over ting vi skulle gjøre, som for eksempel beregne TF-IDF til "queryen" som settes av brukeren, og lette etter en setning i korpuset som ligner mest på inputsetningen ved å beregne "cosine similiarity". <br> Alt i alt, er vi fornøyd med denne delen, siden det tok oss mindre enn 5 min for å løse denne. (VI GIR OSS MULIGHETEN TIL Å SKRYTE AV DET, SIDEN DET TOK OSS LEGIT 5 MIN ;)) 

In [14]:
from oblig2b import RetrievalChatbot
chat = RetrievalChatbot("lotr.en")
print(chat.get_response("Are you Bilbo Baggins ?"))

i 'm sorry , do i know you ? 


<b> Kommentarer: </b>Vi synes denne obligen var den mest spennende av alle. Vi lærte ekstremt mye og jobbet gasnke bra med denne. Takk for rettelsene dette semesteret og ønskere deg, som retter, lykke til videre.